#0. Download Data

In [ ]:
!git clone https://github.com/congnghia0609/ntc-scv.git

Cloning into 'ntc-scv'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 39 (delta 0), reused 4 (delta 0), pack-reused 35
Receiving objects: 100% (39/39), 186.94 MiB | 15.84 MiB/s, done.
Resolving deltas: 100% (9/9), done.
Updating files: 100% (11/11), done.


In [ ]:
# giải nén file
!unzip ./ntc-scv/data/data_test.zip -d ./data
!unzip ./ntc-scv/data/data_train.zip -d ./data
!rm -rf ./ntc-scv

#1. Định dạng lại dữ liệu thành DataFrame

In [ ]:
# data đang ở các dạng file .text, cần định dạng lại data thành các bảng

import os
import pandas as pd
import matplotlib.pyplot as plt

def restructure_data(folder_path):
    examples = []
    for label in os.listdir(folder_path):
        full_path = os.path.join(folder_path, label)
        for file_name in os.listdir(full_path):
            file_path = os.path.join(full_path, file_name)
            with open(file_path, "r", encoding="utf-8") as f:
                lines = f.readlines()
            sentence = " ".join(lines)
            if label == "neg":
                label = 0
            if label == "pos":
                label = 1
            data = {
                'sentence': sentence,
                'label': label
            }
            examples.append(data)
    return pd.DataFrame(examples)

In [ ]:
path_data = {
    'train': './data/data_train/train',
    'valid': './data/data_train/test',
    'test': './data/data_test/test'
}

train_DF = restructure_data(path_data['train'])
val_DF = restructure_data(path_data['valid'])
test_DF = restructure_data(path_data['test'])


In [ ]:
# dữ liệu trước khi được xử lí

train_DF.iloc[:5, :]

,sentence,label
0,Đồ uongs minimum là 50k . khoảng 20 : 30 là có...,1
1,"Giá ở đây hơi chát , nên ăn cũng không thấy ng...",1
2,mát zời lang_thang dậo phố chợt nhớ hàng chè r...,1
3,Mấy anh_em ở công_ty cùng đi ăn ! Thấy quán vắ...,1
4,Mình mua hàng sịn đó các bác ạ ❤ ️ Cái j đắp l...,1


In [ ]:
test_DF.iloc[:5, :]


,sentence,label
0,"Không_gian vô_cùng dễ_thương luôn , phía trên ...",1
1,Ngày_trước mở cơ_sở 2 ở làng sinh_viên hanico ...,1
2,"Không_gian quán rất sáng , mát_mẻ , mới bước v...",1
3,Qua ́ n nă ̀ m ơ ̉ mô ̣ t go ́ c nho ̉ trên tâ...,1
4,Mình ăn cô từ thời mình cấp 2 - 2008 . Bao ngư...,1


In [ ]:
pos_train = (train_DF['label'] == 0).count()
neg_train = (train_DF['label'] == 1).count()
pos_train, neg_train

(30000, 30000)

In [ ]:
pos_test = (test_DF['label'] == 0).count()
neg_test = (test_DF['label'] == 1).count()
pos_test, neg_test

(10000, 10000)

#2. Xử lí dữ liệu

In [ ]:
import re
import string


def preprocess_text(text):
    # remove URLs https://www.
    url_pattern = re.compile(r'https?://\s+\wwww\.\s+')
    text = url_pattern.sub(r" ", text)

    # remove HTML Tags: <>
    html_pattern = re.compile(r'<[^<>]+>')
    text = html_pattern.sub(" ", text)

    # remove puncs and digits
    replace_chars = list(string.punctuation + string.digits)
    for char in replace_chars:
        text = text.replace(char, " ")

    # remove emoji
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r" ", text)

    # normalize whitespace
    text = " ".join(text.split())

    # lowercasing
    text = text.lower()
    return text

In [ ]:
train_DF['sentence'] = [preprocess_text(row['sentence']) for idx, row in train_DF.iterrows()]
val_DF['sentence'] = [preprocess_text(row['sentence']) for idx, row in val_DF.iterrows()]
test_DF['sentence'] = [preprocess_text(row['sentence']) for ind, row in test_DF.iterrows()]


In [ ]:
# dữ liệu sau khi được xử lí

train_DF.iloc[:5, :]

,sentence,label
0,đồ uongs minimum là k khoảng là có múa lửa đã ...,1
1,giá ở đây hơi chát nên ăn cũng không thấy ngon...,1
2,mát zời lang thang dậo phố chợt nhớ hàng chè r...,1
3,mấy anh em ở công ty cùng đi ăn thấy quán vắng...,1
4,mình mua hàng sịn đó các bác ạ cái j đắp lên m...,1


In [ ]:
test_DF.iloc[:5, :]


,sentence,label
0,không gian vô cùng dễ thương luôn phía trên lầ...,1
1,ngày trước mở cơ sở ở làng sinh viên hanico ng...,1
2,không gian quán rất sáng mát mẻ mới bước vào l...,1
3,qua ́ n nă ̀ m ơ ̉ mô ̣ t go ́ c nho ̉ trên tâ...,1
4,mình ăn cô từ thời mình cấp bao người vẫn ăn h...,1


#3. Text Classification using BERT

In [ ]:
!pip install -q -U transformers datasets accelerate evaluate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.0 MB/s eta 0:00:00


###**3.1. Load Dataset**

In [ ]:
from datasets import Dataset, DatasetDict

raw_dataset = DatasetDict({
    'train': Dataset.from_pandas(train_DF),
    'valid': Dataset.from_pandas(val_DF),
    'test': Dataset.from_pandas(test_DF)
    })

In [ ]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 30000
    })
    valid: Dataset({
        features: ['sentence', 'label'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 10000
    })
})

###**3.2. Chuẩn bị dữ liệu cho model**

In [ ]:
from transformers import AutoTokenizer

# sử dụng Tokenizer của bert-base
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=True
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
max_seq_length = 100
max_seq_length = min(max_seq_length, tokenizer.model_max_length)
print(max_seq_length, tokenizer.model_max_length)


sentence = [
    'i go to',
    'i love you'
]

result = tokenizer(
    sentence,
    padding='max_length',
    max_length = max_seq_length,
    truncation = True
)
'''
    input_ids : danh sách các chuỗi token sau khi được tokenizer. 1 chuỗi có cấu trúc
    [CLS]: mã token đánh dấu bắt đầu của chuỗi và quá trình mã hóa
    ....... (các từ trong câu)
    [SEP]: mã token đánh dấu kết thúc 1 chuỗi và quá trình mã hóa

    token_type_ids : Là danh sách các chuỗi token type, sử dụng để phân biệt giữa các phần của câu.
    một chuỗi này có thể chứa các giá trị 0 hoặc 1 để chỉ ra xem token thuộc về câu đầu tiên hay thứ hai trong chuỗi

    attention_mask: Là danh sách các chuỗi attention mask, được sử dụng để chỉ ra mô hình phải chú ý đến token nào và token nào không cần chú ý đến trong quá trình dự đoán.
    Giá trị 1 cho biết mô hình cần chú ý đến token tương ứng, trong khi giá trị 0 cho biết token không cần được chú ý.
'''
result

100 512


{'input_ids': [[101, 1045, 2175, 2000, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 1045, 2293, 2017, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
def preprocess_func(examples):
    result = tokenizer(
        examples['sentence'],
        padding='max_length',
        max_length = max_seq_length,
        truncation = True
    )
    result['label'] = examples['label']
    return result

In [ ]:
# tokenizer cho từng batch dữ liệu

preprocesed_data = raw_dataset.map(
    preprocess_func,
    batched=True,
    desc = "Running tokenizer on dataset"
)

Running tokenizer on dataset:   0%|          | 0/30000 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/10000 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
preprocesed_data

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 30000
    })
    valid: Dataset({
        features: ['sentence', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['sentence', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 10000
    })
})

In [ ]:
from transformers import DataCollatorWithPadding
# tạo đối tượng data_collator để tạo batches
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

###**3.3. Modelling**

In [ ]:
from transformers import AutoConfig, AutoModelForSequenceClassification

num_classes = 2 # số labels để phân loại

config = AutoConfig.from_pretrained(
    model_name,
    num_labels = num_classes,
    finetuning_task='text-classification'
)

model_BERT = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    config=config
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model_BERT

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

###**3.4. Metric**

In [ ]:
import numpy as np
import evaluate


metric = evaluate.load('accuracy')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    position_pred = np.argmax(predictions, axis=1)
    results = metric.compute(predictions=position_pred, references=labels)
    return results

###**3.5. Trainer**

In [ ]:
from transformers import TrainingArguments, Trainer


training_args = TrainingArguments(
    output_dir = 'save_model',
    learning_rate=2e-5,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=10,
    evaluation_strategy="epoch", # đánh giá sau mỗi epoch
    save_strategy='epoch', # lưu model sau mỗi epoch
    load_best_model_at_end =True # lưu model tốt nhất
)

trainer = Trainer(
    model=model_BERT,
    args=training_args,
    train_dataset = preprocesed_data['train'],
    eval_dataset = preprocesed_data['valid'],
    compute_metrics = compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator
)


### **3.6. Training**

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.445672,0.808800
2,No log,0.398510,0.830100
3,0.453400,0.392825,0.836300
4,0.453400,0.395858,0.837500
5,0.323200,0.402308,0.842600
6,0.323200,0.401189,0.845500
7,0.249500,0.427091,0.847600
8,0.249500,0.456626,0.845700
9,0.195900,0.466977,0.844300
10,0.195900,0.483981,0.842800


TrainOutput(global_step=2350, training_loss=0.285355044425802, metrics={'train_runtime': 5333.4089, 'train_samples_per_second': 56.249, 'train_steps_per_second': 0.441, 'total_flos': 1.54166634e+16, 'train_loss': 0.285355044425802, 'epoch': 10.0})

In [ ]:
trainer.evaluate(preprocesed_data["test"])

{'eval_loss': 0.3751769959926605,
 'eval_accuracy': 0.8438,
 'eval_runtime': 55.469,
 'eval_samples_per_second': 180.281,
 'eval_steps_per_second': 1.424,
 'epoch': 10.0}